In [186]:
# Calcula el porcentaje de consumo mensual
# por entidad y el porcentaje que
# corresponde a periodos de 6 meses.

import os

import pandas as pd
import numpy as np

In [187]:
path = "../../../datos/"

path_d = path + "CFE/"
fname = "Consumo_mensual_nacional.csv"

df_0 = pd.read_csv(path_d + fname)
df_0.set_index( "CVE_ENT", inplace = True)

# Convertimos de texto a fecha.
df = df_0.drop(columns = "NOM_ENT").transpose()
df.index = pd.to_datetime(df.index,
    format = "%y-%b")

# Calculamos el consumo total anual.
df_y = df.groupby(df.index.year).sum()

# Calculamos el porcentaje mensual.
for i in df.itertuples():
    df.loc[i[0]] = ( i[1:] / df_y.loc[
        df_y.index == i[0].year].iloc[0] )

# Guardamos el porcentaje mensual.
df.transpose().reset_index().to_csv( 
    path_d + "Porcentaje_mensual.csv",
    index = False )

In [193]:
df_i = []

for j in range(2, 6):
    # Escoge 6 meses consecutivos.
    df_i.append( df[ df.index.month.isin(
        range( j, j + 6 ) ) ] )
    # Suma los 6 meses consecutivos.
    df_i[j - 2] = df_i[j - 2].groupby(
        df_i[j - 2].index.year).sum()
    # Hace que la entidad y el año sean columnas.
    df_i[j - 2] = ( df_i[j - 2].transpose()
        .reset_index().melt( id_vars = "CVE_ENT",
        var_name = "Año", value_name = j ) )
    # Crea una columna con valor único para el merge.
    df_i[j - 2].reset_index(inplace = True)

# Une todas las tablas.
df_p = df_i[0]
for j in range(3, 6):
    df_i[j - 2].drop(columns = ["CVE_ENT", "Año"], inplace = True)
    df_p = pd.merge(df_p, df_i[j - 2], on = "index")

# Quita la columna con valor único.
df_p.drop(columns = "index", inplace = True)

# Calcula el porcentaje promedio.
df_mean = df_p.groupby("CVE_ENT").mean().reset_index()
df_mean["Año"] = "mean"
df_p = df_p.append(df_mean)

df_p.reset_index().to_csv( path_d
    + "Porcentaje_verano.csv",
    index = False )